## Lab 5 Homework: Image Classification

### Kurt Wedegaertner |  CEE 5003  |  03/27/23

# 1. Call geemap

In [1]:
import ee
ee.Initialize()

import geemap
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

# 2. Determine a POI and call Landsat Imagery

In [2]:
point = ee.Geometry.Point([-111.364273, 41.856689])
# point = Bear Lake Region ofUTah/Idaho

landsat = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(point) \
    .filterDate('2021-05-01', '2021-07-31') \
    .sort('CLOUD_COVER') \
    .first() \
    .select('SR_B[1-7]')

landsat = landsat.multiply(0.0000275).add(-0.2)


vis_params = {
    'min': 0,
    'max': 0.2,
    'bands': ['SR_B5', 'SR_B4', 'SR_B3']
}

Map.centerObject(point, 8)
Map.addLayer(landsat, vis_params, 'false color composite')

Map

Map(center=[41.856689, -111.364273], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HB…

# 3. Create a NDVI Layer in order to Determine Differences in Land Cover

In [3]:
img_name ='ndvi'
ndvi=landsat.normalizedDifference(['SR_B5', 'SR_B4'])

vis_params = {
  'min': -0.2,
  'max': 1.0,
  'palette': ['blue','white','brown','yellow', 'lime', 'green','navy']}

Map.addLayer(ndvi,vis_params,img_name)

colors = vis_params['palette']
vmin = vis_params['min']
vmax = vis_params['max']

Map.add_colorbar_branca(colors=colors, vmin=vmin, vmax=vmax, layer_name=img_name)

Map

Map(center=[41.856689, -111.364273], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HB…

# 4. Make a training dataset to recognize differences in land cover

### Bare Soil

In [24]:
# Find an area of bare soil and define it using the geometry rectangle
polygon = ee.Geometry.Polygon([
                   [-110.622, 41.579],
                   [-110.614, 41.579],
                   [-110.614, 41.570],
                   [-110.622, 41.571]]);
# Labels the area as 'baresoil'
baresoil = ee.Feature(polygon, {'class': 2, 'name': 'bare soil'});
Map.addLayer(baresoil, {'fill_color':'yellow', 'outline': 1}, name='baresoil')

# Bare soil 2
polygon = ee.Geometry.Polygon([
                   [-112.3218, 41.3827],
                   [-112.2814, 41.3829],
                   [-112.2831, 41.3640],
                   [-112.3226, 41.3637]]);
# Labels the area as 'baresoil_2'
baresoil_2 = ee.Feature(polygon, {'class': 2, 'name': 'bare soil_2'});
Map.addLayer(baresoil_2, {'fill_color':'yellow', 'outline': 1}, name='baresoil_2')




Map

Map(bottom=1559343.0, center=[41.86936908105971, -111.6152572631836], controls=(WidgetControl(options=['positi…

### Vegetation

In [26]:
# Find an area of vegetation and define it using the geometry rectangle
polygon = ee.Geometry.Polygon([
                 [-111.667, 41.731],
                 [-111.627, 41.730],
                 [-111.626, 41.711],
                 [-111.665, 41.711]]);
# Labels the area as 'vegetation'
vegetation = ee.Feature(polygon, {'class': 1, 'name': 'vegetation'});
Map.addLayer(vegetation, {'fill_color':'green', 'outline': 1}, name='vegetation')

# Vegetation 2
polygon = ee.Geometry.Polygon([
                 [-111.6350, 41.8752],
                 [-111.6118, 41.8748],
                 [-111.6118, 41.8645],
                 [-111.6272, 41.8654]]);
# Labels the area as 'vegetation_2'
vegetation_2 = ee.Feature(polygon, {'class': 1, 'name': 'vegetation_2'});
Map.addLayer(vegetation_2, {'fill_color':'green', 'outline': 1}, name='vegetation_2')
Map

Map(bottom=1559343.0, center=[41.86936908105971, -111.6152572631836], controls=(WidgetControl(options=['positi…

### Water

In [28]:
# Find an area of water and define it using the geometry rectangle
polygon = ee.Geometry.Polygon([
                   [-111.305, 42.077],
                   [-111.284, 42.076],
                   [-111.285, 42.064],
                   [-111.303, 42.065]]);

# Labels the area as 'water'
water = ee.Feature(polygon, {'class': 0, 'name': 'water'});
Map.addLayer(water, {'fill_color':'blue', 'outline': 1}, name='water')

# water 2
polygon = ee.Geometry.Polygon([
                   [-112.6751, 41.1789],
                   [-112.5872, 41.1786],
                   [-112.5882, 41.1130],
                   [-112.6758, 41.1127]]);

# Labels the area as 'water_2'
water_2 = ee.Feature(polygon, {'class': 0, 'name': 'water_2'});
Map.addLayer(water_2, {'fill_color':'blue', 'outline': 1}, name='water_2')
Map

Map(bottom=392879.0, center=[41.144794118797314, -112.55321502685548], controls=(WidgetControl(options=['posit…

# 5. Merge Features into a Feature Collection and Specify the Bands of the Landsat Composite to be Used as Predictors

In [29]:
# add additional features to this if you create more to strengthen the model
trainingFeatures = ee.FeatureCollection([water, vegetation, baresoil, baresoil_2, vegetation_2, water_2])

predictionBands = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5' ,'SR_B6','SR_B7']
print(trainingFeatures.first().getInfo())

classifierTraining = landsat.select(predictionBands).sampleRegions(
      collection= trainingFeatures, 
      properties= ['class'], 
      scale= 30
    );

{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-111.305, 42.077], [-111.303, 42.065], [-111.285, 42.064], [-111.284, 42.076], [-111.305, 42.077]]]}, 'id': '0', 'properties': {'class': 0, 'name': 'water'}}


In [30]:
# ranonly splis data into 60% for training, and 40% for testing
trainingTesting = classifierTraining.randomColumn('random',111009);

training = trainingTesting.filter(ee.Filter.lt('random', 0.66));

testing = trainingTesting.filter(ee.Filter.gte('random', 0.66));

# 6. Train Models

### CART Model

In [31]:
#hyperparameter to tune
leaf_val=1

cartclassifier = ee.Classifier.smileCart(minLeafPopulation=leaf_val).train(
      features= training, 
      classProperty= 'class', 
      inputProperties= predictionBands
    );

In [32]:
# Make Predictions
cartClasifficationImage = landsat.select(predictionBands).classify(cartclassifier);
# Visualize
Map.addLayer(cartClasifficationImage, {'min': 0, 'max': 2,
                                   'palette':['blue', 'green','yellow']},'CART classification');
Map

Map(bottom=98250.0, center=[41.34588656996289, -112.05642700195314], controls=(WidgetControl(options=['positio…

### Random Forest Model

In [33]:
# hyperparameter to tune
trees_val=13

rfClassification = ee.Classifier.smileRandomForest(numberOfTrees=trees_val, seed=111009).train(
      features= training, 
      classProperty= 'class', 
      inputProperties= predictionBands
    )

In [34]:
# Performs the RF regression 
rfClassificationImage = landsat.select(predictionBands).classify(rfClassification);
    
# Visualize
Map.addLayer(rfClassificationImage,  {'min': 0, 'max': 2,
                                   'palette':['blue','green', 'yellow']}, 'RF classification');

Map

Map(bottom=98250.0, center=[41.34588656996289, -112.05642700195314], controls=(WidgetControl(options=['positio…

### Support Vector Machine

In [35]:
# hyperparameter to tune
gamma_val =0.1

# Create an SVM classifier with custom parameters.
svClassification = ee.Classifier.libsvm(svmType='C_SVC',kernelType='RBF',gamma=gamma_val).train(
      features= training, 
      classProperty= 'class', 
      inputProperties= predictionBands
    )

In [37]:
# Performs the RF regression
svClassificationImage = landsat.select(predictionBands).classify(svClassification);
    
# Visualize
Map.addLayer(svClassificationImage,{'min': 0, 'max': 2,
                                   'palette':['blue', 'green','yellow']}, 'SV CLassification');
Map

Map(bottom=390470.0, center=[41.764653945369396, -111.82743072509766], controls=(WidgetControl(options=['posit…

# 7. Perform an Accuracy Assessment 

### CART Assessment

In [38]:
# Perform the CART classification on the test set

test=testing.classify(cartclassifier)
# print(test.first().getInfo())
# // Get a confusion matrix representing expected accuracy.
testAccuracy = test.errorMatrix('class', 'classification');

import numpy as np
errormaxtrix=np.array(testAccuracy.array().getInfo())

print(testAccuracy.name());
print(errormaxtrix)
print('Overall Accuracy:', testAccuracy.accuracy().getInfo());
print('Producers Accuracy:', testAccuracy.producersAccuracy().getInfo());
print('Consumers Accuracy:', testAccuracy.consumersAccuracy().getInfo());
print('Kappa:', testAccuracy.kappa().getInfo());

ConfusionMatrix
[[21109     0     0]
 [    0  3312     0]
 [    0     0  2905]]
Overall Accuracy: 1
Producers Accuracy: [[1], [1], [1]]
Consumers Accuracy: [[1, 1, 1]]
Kappa: 1


### Random Trees Assessment

In [39]:
# Perform the RF classification on the test set

test=testing.classify(rfClassification)
# print(test.first().getInfo())
# Get a confusion matrix representing expected accuracy.
testAccuracy = test.errorMatrix('class', 'classification');

errormaxtrix=np.array(testAccuracy.array().getInfo())

print(testAccuracy.name());
print(errormaxtrix)
print('Overall Accuracy:', testAccuracy.accuracy().getInfo());
print('Producers Accuracy:', testAccuracy.producersAccuracy().getInfo());
print('Consumers Accuracy:', testAccuracy.consumersAccuracy().getInfo());
print('Kappa:', testAccuracy.kappa().getInfo());

ConfusionMatrix
[[21109     0     0]
 [    0  3312     0]
 [    0     0  2905]]
Overall Accuracy: 1
Producers Accuracy: [[1], [1], [1]]
Consumers Accuracy: [[1, 1, 1]]
Kappa: 1


### SVM Assessment

In [40]:
# Perform the SVR classification on the test set

test=testing.classify(svClassification)
# print(test.first().getInfo())
# Get a confusion matrix representing expected accuracy.
testAccuracy = test.errorMatrix('class', 'classification');

errormaxtrix=np.array(testAccuracy.array().getInfo())

print(testAccuracy.name());
print(errormaxtrix)
print('Overall Accuracy:', testAccuracy.accuracy().getInfo());
print('Producers Accuracy:', testAccuracy.producersAccuracy().getInfo());
print('Consumers Accuracy:', testAccuracy.consumersAccuracy().getInfo());
print('Kappa:', testAccuracy.kappa().getInfo());

ConfusionMatrix
[[21109     0     0]
 [    0  3312     0]
 [    0     0  2905]]
Overall Accuracy: 1
Producers Accuracy: [[1], [1], [1]]
Consumers Accuracy: [[1, 1, 1]]
Kappa: 1
